In [ ]:
!python -m venv .venv

In [ ]:
import matplotlib.pyplot as plt
from model.yolo.ultralytics.models.yolo.model import YOLO

In [6]:
!yolo obb train \
    model=/home/chisphung/ALPR_Fisheye/configs/vertex-yolo.yaml \
    data=/home/chisphung/ALPR_Fisheye/dataset/dataset.yaml \
    imgsz=640 \
    epochs=300 \
    batch=16 \
    device=0 \
    project=/home/chisphung/ALPR_Fisheye/runs \
    name=vertex_obb_train

New https://pypi.org/project/ultralytics/8.3.204 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.202 🚀 Python-3.12.3 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 3904MiB)
Ultralytics 8.3.202 🚀 Python-3.12.3 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 3904MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/chisphung/ALPR_Fisheye/dataset/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_r

In [1]:
import os 

tracks = os.listdir('/home/chisphung/ALPR_Fisheye/testing_label')

In [4]:
import shutil

In [ ]:
import os, glob
from PIL import Image

IMG_DIR = "/kaggle/working/UFPR-ALPR_dataset/validation"   # where images live
LBL_DIR = "/kaggle/working/UFPR-ALPR_dataset/validation"   # where labels live
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}

# If you ALSO want to convert detect-format YOLO (class cx cy w h) into axis-aligned OBB,
# set this True. Otherwise we leave detect labels unchanged.
MAKE_OBB_FROM_DETECT = False

def find_image(lbl_path):
    stem = os.path.splitext(os.path.basename(lbl_path))[0]
    # labels may be in a separate folder; we scan IMG_DIR for a matching basename
    for root, _, files in os.walk(IMG_DIR):
        for f in files:
            n, e = os.path.splitext(f)
            if n == stem and e.lower() in IMG_EXTS:
                return os.path.join(root, f)
    return None

def clamp01(v): 
    return 0.0 if v < 0 else 1.0 if v > 1 else v

converted = skipped = 0
for lbl_path in glob.glob(os.path.join(LBL_DIR, "**/*.txt"), recursive=True):
    img_path = find_image(lbl_path)
    if not img_path:
        print(f"[skip] no image for {lbl_path}")
        skipped += 1
        continue

    with Image.open(img_path) as im:
        W, H = im.size

    out = []
    changed = False
    with open(lbl_path, "r", encoding="utf-8") as f:
        for raw in (ln.strip() for ln in f if ln.strip()):
            parts = raw.split()
            if len(parts) < 2: 
                continue
            cls = parts[0]
            nums = parts[1:]

            # ---- Case A: OBB (class + 8 coords)
            if len(nums) == 8:
                try:
                    vals = [float(x) for x in nums]
                except:
                    print(f"[warn] non-numeric OBB in {lbl_path}: {raw}")
                    continue

                # Decide if pixel or already normalized:
                # If any coord > 1.5, we treat as pixel coordinates.
                if any(abs(v) > 1.5 for v in vals):
                    # normalize x by W and y by H, preserving order x1 y1 x2 y2 ...
                    norm = []
                    for i, v in enumerate(vals):
                        if i % 2 == 0:   # x
                            norm.append(clamp01(v / W))
                        else:            # y
                            norm.append(clamp01(v / H))
                    out.append(" ".join([cls] + [f"{v:.6f}" for v in norm]))
                    changed = True
                else:
                    # Already normalized [0,1] — keep
                    out.append(raw)

            # ---- Case B: Detect (class + cx cy w h). By default we leave as-is.
            elif len(nums) == 4:
                if not MAKE_OBB_FROM_DETECT:
                    out.append(raw)  # unchanged
                else:
                    # Optional: convert detect → axis-aligned OBB
                    try:
                        cx, cy, w, h = map(float, nums)
                    except:
                        print(f"[warn] non-numeric detect in {lbl_path}: {raw}")
                        continue
                    # If looks like pixels, normalize first
                    if max(abs(cx), abs(cy), abs(w), abs(h)) > 1.5:
                        cx, cy, w, h = cx / W, cy / H, w / W, h / H
                    w = max(1e-6, min(1.0, w)); h = max(1e-6, min(1.0, h))
                    cx = clamp01(cx); cy = clamp01(cy)
                    x1, y1 = cx - w/2, cy - h/2
                    x2, y2 = cx + w/2, cy - h/2
                    x3, y3 = cx + w/2, cy + h/2
                    x4, y4 = cx - w/2, cy + h/2
                    out.append(f"{cls} {x1:.6f} {y1:.6f} {x2:.6f} {y2:.6f} {x3:.6f} {y3:.6f} {x4:.6f} {y4:.6f}")
                    changed = True

            else:
                print(f"[warn] Unexpected field count ({len(nums)}) in {lbl_path}: {raw}")

    if not out:
        skipped += 1
        continue

    # Backup once
    bak = lbl_path + ".bak"
    if not os.path.exists(bak):
        with open(bak, "w", encoding="utf-8") as b:
            with open(lbl_path, "r", encoding="utf-8") as src:
                b.write(src.read())

    with open(lbl_path, "w", encoding="utf-8") as f:
        f.write("\n".join(out) + "\n")

    converted += 1 if changed else 0

print(f"Done. Files_changed={converted}, Files_unchanged={len(list(glob.iglob(os.path.join(LBL_DIR, '**/*.txt'), recursive=True))) - converted}, Skipped={skipped}")
